In [27]:
import sqlite3
import pandas as pd

import os, sys
rootpath = os.path.join(os.getcwd(), '.')
sys.path.append(rootpath)
from src.utils import *
# reimport modules if they change
%load_ext autoreload
%autoreload 2

/Users/johannes/git/projects/sahel_finite-observation-dynamic-range/0_update/.
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
# connect to the database (create file if not exists)
con = sqlite3.connect('./simulations.db')
# create a cursor object
cur = con.cursor()

## Simulations

In [14]:
# create table that does not allow duplicates
cur.execute("""
CREATE TABLE IF NOT EXISTS simulations (
            simulation_id integer PRIMARY KEY,
            N int NOT NULL,
            K int NOT NULL,
            lambda float NOT NULL,
            mu float NOT NULL,
            h float NOT NULL,
            seed int NOT NULL,
            raw_file TEXT NOT NULL
            )
""")
cur.execute("CREATE UNIQUE INDEX IF NOT EXISTS parameters ON simulations (N,K,lambda,mu,h,seed);")
# can be dropped with cur.execute("DROP INDEX parameters;") if needs to be redesigned
con.commit()

In [25]:
# show the simulations
# cur.execute("SELECT * FROM simulations")
# print(cur.fetchall())
simulations = pd.read_sql_query("SELECT * FROM simulations", con)
print(simulations)

   simulation_id     N    K  lambda   mu     h  seed raw_file
0              1  1000  100     0.9  0.2  0.01  1000     path


## Beta Approximation

In [16]:
# create table that does not allow duplicates
cur.execute("""
CREATE TABLE IF NOT EXISTS beta_approximations (
            simulation_id INTEGER NOT NULL UNIQUE,
            a float NOT NULL,
            b float NOT NULL,
            loc float NOT NULL,
            scale float NOT NULL,
            FOREIGN KEY (simulation_id) REFERENCES simulations (simulation_id)
            )
""")
con.commit()

In [17]:
# show state of approximation simulations
cur.execute("SELECT * FROM beta_approximations")
print(cur.fetchall())

[]


## Neural Network Interpolation

## Clean up 

In [24]:
know_what_you_do = True
if know_what_you_do:
    # add previous simulations and their locations to the database
    params={'N':1000, 'K':100, 'lambda':0.9, 'mu':0.2, 'h':0.01, 'seed':1000, 'raw_file':'path'}
    insert_from_dict(con, cur, 'simulations', params)
    pass

In [11]:
con.close()